In [1]:
import pandas as pd
import re

# File paths
plans_path = "../Data/plans.xlsx"
structure_sets_path = "../Data/structure_sets.xlsx"
dosimetrics_path = "../Data/dosimetrics.xlsx"

# Load data
plans_df = pd.read_excel(plans_path)
structure_sets_df = pd.read_excel(structure_sets_path)
dosimetrics_df = pd.read_excel(dosimetrics_path)

# Add prefixes to avoid column name conflicts
plans_df = plans_df.add_prefix("plans_")
structure_sets_df = structure_sets_df.add_prefix("structures_")
dosimetrics_df = dosimetrics_df.add_prefix("dosimetrics_")

# Extract core key from filenames for RP-RD merging (strict one-to-one mapping)
def extract_core_key(file_path):
    match = re.search(r'R[PD]\.(\d+)\.(.*?)\.dcm', file_path)
    return f"{match.group(1)}.{match.group(2)}" if match else None

plans_df["core_key"] = plans_df["plans_file_path"].apply(extract_core_key)
dosimetrics_df["core_key"] = dosimetrics_df["dosimetrics_file_path"].apply(extract_core_key)

# Ensure one-to-one merge between RP and RD
merged_df = plans_df.merge(
    dosimetrics_df, on="core_key", suffixes=("_plans", "_dosimetrics"), how="inner"
)

# Merge structure sets using patient_id (one-to-many relationship)
final_merged_df = merged_df.merge(
    structure_sets_df, left_on="plans_patient_id", right_on="structures_patient_id", 
    suffixes=("", "_structures"), how="left"
)

# Save final merged dataset
final_merged_df.to_excel("../Data/metadata.xlsx", index=False)


In [8]:
import pydicom

# Function to extract structures from RS file
def extract_structures_from_rs(rs_file_path):
    try:
        ds = pydicom.dcmread(rs_file_path)
        if "StructureSetROISequence" in ds:
            structures = {roi.ROIName for roi in ds.StructureSetROISequence}
            print(f"[RS] Extracted {len(structures)} structures from {rs_file_path}")
            return structures, ds.FrameOfReferenceUID
        print(f"[RS] No structures found in {rs_file_path}")
        return set(), ds.FrameOfReferenceUID
    except Exception as e:
        print(f"[ERROR] Failed to read RS file {rs_file_path}: {e}")
        return set(), None

# Function to extract Frame of Reference UID and doses to CTVs from RD file
def extract_frame_of_reference_and_ctv_doses(rd_file_path):
    try:
        ds = pydicom.dcmread(rd_file_path)
        frame_of_reference_uid = ds.FrameOfReferenceUID if "FrameOfReferenceUID" in ds else None

        # Extract doses assigned to CTVs
        ctv_doses = {}
        if "DVHSequence" in ds:
            for dvh in ds.DVHSequence:
                if hasattr(dvh, "DVHReferencedROISequence"):
                    for ref_roi in dvh.DVHReferencedROISequence:
                        roi_number = ref_roi.ReferencedROINumber
                        dose = dvh.DVHMaximumDose if hasattr(dvh, "DVHMaximumDose") else None
                        if dose:
                            ctv_doses[f"CTV_{roi_number}_dose"] = dose

        print(f"[RD] Extracted Frame of Reference UID: {frame_of_reference_uid} from {rd_file_path}")
        print(f"[RD] Extracted {len(ctv_doses)} CTV doses from {rd_file_path}")

        return frame_of_reference_uid, ctv_doses
    except Exception as e:
        print(f"[ERROR] Failed to read RD file {rd_file_path}: {e}")
        return None, {}

# Process each RD-RS pair in the merged dataset
matching_files = []
total_pairs = 0
matched_pairs = 0

print("\n[INFO] Checking RD-RS structure consistency using Frame of Reference UID...")

for index, row in final_merged_df.iterrows():
    rd_file = row.get("dosimetrics_file_path")
    rs_file = row.get("structures_file_path")
    
    if pd.isna(rd_file) or pd.isna(rs_file):
        print(f"[SKIP] Missing RD or RS file for row {index}. Skipping.")
        continue
    
    total_pairs += 1
    print(f"\n[PROCESSING] Comparing RD: {rd_file} with RS: {rs_file}")

    rd_for_uid, rd_ctv_doses = extract_frame_of_reference_and_ctv_doses(rd_file)
    rs_structures, rs_for_uid = extract_structures_from_rs(rs_file)
    
    # Check if Frame of Reference UID matches
    if rd_for_uid and rs_for_uid and rd_for_uid == rs_for_uid:
        print(f"[MATCH] RD and RS have matching Frame of Reference UID: {rd_for_uid}.")
        row_data = row.copy()
        row_data["available_structures"] = ", ".join(rs_structures)  # Store structures as comma-separated list
        row_data.update(rd_ctv_doses)  # Add CTV dose data dynamically
        matching_files.append(row_data)
        matched_pairs += 1
    else:
        print(f"[NO MATCH] RD and RS have different Frame of Reference UIDs (RD: {rd_for_uid}, RS: {rs_for_uid}).")

# Convert the filtered dataset to a DataFrame
filtered_df = pd.DataFrame(matching_files)

# Save and display the result
filtered_df.to_excel("../Data/metadata.xlsx", index=False)



[INFO] Checking RD-RS structure consistency using Frame of Reference UID...

[PROCESSING] Comparing RD: ../DICOM/172543/RD.172543.odbytnica.dcm with RS: ../DICOM/172543/RS.172543.Auto__Miednica.0001.dcm
[RD] Extracted Frame of Reference UID: 1.3.12.2.1107.5.1.7.128362.30000024070410270698700000093 from ../DICOM/172543/RD.172543.odbytnica.dcm
[RD] Extracted 0 CTV doses from ../DICOM/172543/RD.172543.odbytnica.dcm
[RS] Extracted 11 structures from ../DICOM/172543/RS.172543.Auto__Miednica.0001.dcm
[MATCH] RD and RS have matching Frame of Reference UID: 1.3.12.2.1107.5.1.7.128362.30000024070410270698700000093.

[PROCESSING] Comparing RD: ../DICOM/172543/RD.172543.odbytnica.dcm with RS: ../DICOM/172543/RS.172543.Auto__Miednica.0002.dcm
[RD] Extracted Frame of Reference UID: 1.3.12.2.1107.5.1.7.128362.30000024070410270698700000093 from ../DICOM/172543/RD.172543.odbytnica.dcm
[RD] Extracted 0 CTV doses from ../DICOM/172543/RD.172543.odbytnica.dcm
[RS] Extracted 34 structures from ../DICOM/17

In [10]:
# Filter the dataset to retain only cases where "PTV" is present in the available structures list
filtered_df_ptv = filtered_df[filtered_df["available_structures"].str.contains("PTV", case=False, na=False)]

# Save and display the filtered dataset
filtered_df_ptv.to_excel("../Data/metadata.xlsx", index=False)

# Summary output
print(f"[INFO] Filtered dataset contains {len(filtered_df_ptv)} cases with PTV structures.")

[INFO] Filtered dataset contains 243 cases with PTV structures.


In [11]:
filtered_df_ptv

,plans_file_path,plans_plan_name,plans_plan_date,plans_reference_dose_name,plans_reference_dose,plans_approval,plans_CT_series,plans_CT_study,plans_patient_id,plans_patient_dob,...,dosimetrics_processing_timestamp,dosimetrics_file_pattern,structures_file_path,structures_CT_series,structures_CT_study,structures_approval,structures_patient_id,structures_processing_timestamp,structures_file_pattern,available_structures
1,../DICOM/172543/RP.172543.odbytnica.dcm,odbytnica,20240821,PTV1,50.0,APPROVED,1.2.246.352.71.2.846132777028.18433625.2024070...,1.2.246.352.76.1.846132777028.364035.241864723...,172543,19540507.0,...,2025-02-28T04:29:23.642688,RD*.dcm,../DICOM/172543/RS.172543.Auto__Miednica.0002.dcm,1.2.246.352.71.2.846132777028.18439874.2024071...,1.2.246.352.76.1.846132777028.364035.241864723...,APPROVED,172543,2025-02-28T04:26:56.506875,RS*.dcm,"gl, Odbytnica - PTV, podkladka, Jelita - PTV, ..."
2,../DICOM/304962/RP.304962.Odb +.dcm,Odb +,20230915,odb,50.0,APPROVED,1.2.246.352.71.2.846132777028.17504783.2023070...,1.3.12.2.1107.5.1.4.49619.30000023070307470284...,304962,19591026.0,...,2025-02-28T04:29:23.642688,RD*.dcm,../DICOM/304962/RS.304962.CT_1.0001.dcm,1.2.246.352.71.2.846132777028.17502857.2023070...,1.3.12.2.1107.5.1.4.49619.30000023070307470284...,APPROVED,304962,2025-02-28T04:26:56.506875,RS*.dcm,"gl, Odbytnica - PTV, podkladka, Jelita - PTV, ..."
3,../DICOM/363068/RP.363068.odb.dcm,odb,20230426,PTV1,60.0,APPROVED,1.2.246.352.71.2.846132777028.17189896.2023030...,1.3.12.2.1107.5.1.4.49619.30000023030207475281...,363068,19501105.0,...,2025-02-28T04:29:23.642688,RD*.dcm,../DICOM/363068/RS.363068.CT_1.0001.dcm,1.2.246.352.71.2.846132777028.17190337.2023030...,1.3.12.2.1107.5.1.4.49619.30000023030207475281...,APPROVED,363068,2025-02-28T04:26:56.506875,RS*.dcm,"gl, Odbytnica - PTV, podkladka, Jelita - PTV, ..."
4,../DICOM/371163/RP.371163.odb.dcm,odb,20230217,PTV1,50.0,APPROVED,1.2.246.352.71.2.846132777028.17032888.2022123...,1.3.12.2.1107.5.1.4.49619.30000022122007420785...,371163,19480114.0,...,2025-02-28T04:29:23.642688,RD*.dcm,../DICOM/371163/RS.371163.CT_1.0001.dcm,1.2.246.352.71.2.846132777028.17033413.2022123...,1.3.12.2.1107.5.1.4.49619.30000022122007420785...,APPROVED,371163,2025-02-28T04:26:56.506875,RS*.dcm,"gl, Odbytnica - PTV, Jelita - PTV, Rdzen, Jeli..."
5,../DICOM/377007/RP.377007.RAP1.dcm,RAP1,20190926,PTV 1,50.0,APPROVED,1.2.246.352.71.2.846132777028.14782935.2019080...,1.3.12.2.1107.5.1.4.49619.30000019080107404148...,377007,19470716.0,...,2025-02-28T04:29:23.642688,RD*.dcm,../DICOM/377007/RS.377007.CT_1.0001.dcm,1.2.246.352.205.4843651491561318283.7818544043...,1.3.12.2.1107.5.1.4.49619.30000019080107404148...,APPROVED,377007,2025-02-28T04:26:56.506875,RS*.dcm,"gl, PTV 1, PTV2-PTV1, Opuszka Pracia, Pecherz,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364,../DICOM/479238/RP.479238.Odb+w H.dcm,Odb+w H,20240821,odb+w,NaN,APPROVED,1.2.246.352.71.2.846132777028.18418428.2024070...,1.3.12.2.1107.5.1.4.49619.30000024062807373600...,479238,19661116.0,...,2025-02-28T04:29:23.642688,RD*.dcm,../DICOM/479238/RS.479238.CT_1.0001.dcm,1.2.246.352.205.4705975484960778407.1209330727...,1.3.12.2.1107.5.1.4.49619.30000024062807373600...,APPROVED,479238,2025-02-28T04:26:56.506875,RS*.dcm,"gl, Odbytnica - PTV, Jelita - PTV, Rdzen, Jeli..."
365,../DICOM/479297/RP.479297.odbytnica1+.dcm,odbytnica1+,20240823,PTV1,54.0,APPROVED,1.2.246.352.71.2.846132777028.18418569.2024070...,1.2.246.352.76.1.846132777028.363653.241804718...,479297,19510411.0,...,2025-02-28T04:29:23.642688,RD*.dcm,../DICOM/479297/RS.479297.Auto__Miednica.0001.dcm,1.2.246.352.71.2.846132777028.18418464.2024070...,1.2.246.352.76.1.846132777028.363653.241804718...,APPROVED,479297,2025-02-28T04:26:56.506875,RS*.dcm,"gl, Odbytnica - PTV, Jelita - PTV, dod44, Rdze..."
367,../DICOM/480007/RP.480007.Odb+W.dcm,Odb+W,20240830,odb+w,NaN,APPROVED,1.2.246.352.71.2.846132777028.18466631.2024071...,1.3.12.2.1107.5.1.4.49619.